<html>
    <h1 align='center'><b>
    <img src='https://media3.giphy.com/media/KAq5w47R9rmTuvWOWa/giphy.gif' width=100 height=100>
    <font color='sky blue' face='lucida handwriting'size=300>Py</font><font color='yellow' face='lucida handwriting' size=300>Tech</font></b></h1>
   

<html>
    <h1><i><font face='Courier' size = 10>Importing Libraries</font></i></h1>

In [ ]:
import numpy as np 
import plotly.express as px,plotly.graph_objs as go
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

<html>
    <h1><i><font face='Courier' size = 10>Data Cleaning</font></i></h1>

In [ ]:
df

<html>
    <h1><font face='Courier'>We will keep only year from date because month and date both barely make a difference in Price</font></h1>

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['year'] = df['date'].apply(lambda date:date.year)

In [ ]:
df.drop(['date'],axis=1,inplace=True)

In [ ]:
px.defaults.color_continuous_scale
px.scatter(df,x='long',y='lat',color='price',color_continuous_scale=px.colors.sequential.Emrld)

<html>
    <h1><font face='Courier'>Zipcode and id also don't matter much</font></h1>

In [ ]:
df= df.drop(['zipcode','id'],axis=1)

<html>
    <h1><font face='Courier'>I am using only those who have price under 3 millon,because most houses are in this range,but you can try with over 4 or 5 million but if you take full dataset then a few houses are be far off in 6 - 7 million which will make  a big difference in predictions</font></h1>

In [ ]:
df2 = df[df['price']<=3000000]

In [ ]:
df2.describe()

In [ ]:
X = df2.drop('price',axis=1)
y = df2['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

<html>
    <h1><i><font face='Courier' size = 10>Scaling the data and Making the Model</font></i></h1>

In [ ]:
scale = MinMaxScaler()

In [ ]:
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [ ]:
lg = LinearRegression()

<html>
    <h1><i><font face='Courier' size = 10>Linear Regression</font></i></h1>

In [ ]:
lg.fit(X_train,y_train)

In [ ]:
pred1 = lg.predict(X_test)

In [ ]:
pred1 = pd.Series(pred1)

In [ ]:
pred1.index = y_test.index

<html>
    <h1><i><font face='Courier' size = 10>Neural Network</font></i></h1>

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

<html>
    <h1><i><font face='Courier' size = 10>Prediction and Evaluation</font></i></h1>

In [ ]:
model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=1000)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

<html>
    <h1><font face='Courier'>Prediction</font></h1>

In [ ]:
predictions = model.predict(X_test)

In [ ]:
a=[]
for i in range(len(predictions)):
    a.append(predictions[i][0])

In [ ]:
pred = pd.Series(a)

In [ ]:
pred.index = y_test.index

In [ ]:
p=pd.concat([pred,pred1,y_test],axis=1)

In [ ]:
p.columns=['Neural Network','Linear Regression','Original']

In [ ]:
p['Neural net score']=None
for i in p.index:

    
    if p['Original'][i] >= p['Neural Network'][i]:
        p['Neural net score'][i] = p['Original'][i] - p['Neural Network'][i]
    else:
        p['Neural net score'][i] = p['Neural Network'][i] - p['Original'][i] 

In [ ]:
p['linear score']=None
for i in p.index:
    if p['Original'][i] >= p['Linear Regression'][i]:
        p['linear score'][i] = p['Original'][i] - p['Linear Regression'][i]
    else:
        p['linear score'][i] = p['Linear Regression'][i] - p['Original'][i] 

In [ ]:
linear_score = np.array(p['linear score'])

In [ ]:
Neural_score = np.array(p['Neural net score'])

<html>
    <h1><font face='Courier'>Checking the Average loss<br>
        the lesser the better</font></h1>

In [ ]:
np.average(Neural_score)

In [ ]:
np.average(linear_score)

<html>
    <h1><font face='Courier'>Neural ntwork came out better but if you decrease epochs, say 400 epochs then linear regression will give better outcome</font></h1>

In [ ]:
p

<html>
    <h1><font face='Courier'>In the Graph below the closer the points to the red line the higher accuracy you get</font></h1>

In [ ]:
plt.title('Linear Regression')
plt.scatter(y_test,pred1)
plt.plot(y_test,y_test,'r')


in this graph start was good but as the value gets higher points tend to go near X-axis

In [ ]:
plt.title('Neural Network')
plt.scatter(y_test,predictions)
plt.plot(y_test,y_test,'r')

in this graph points steadily go away fom the line but its not very bad